In [12]:
import warnings
warnings.filterwarnings("ignore")

In [13]:
import pandas as pd
from sklearn.metrics import (
    make_scorer,
    mean_absolute_error,
    mean_squared_error,
    r2_score
)
from sklearn.model_selection import cross_validate, StratifiedKFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import  MLPRegressor
from sklearn.svm import SVR
import numpy as np

In [14]:
def root_mean_squared_error(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

METRICS_REGRESSION = {
    "MAE": make_scorer(mean_absolute_error),
    "MSE": make_scorer(mean_squared_error),
    "RMSE": make_scorer(root_mean_squared_error),
    "R2": make_scorer(r2_score)
}

In [15]:
df = pd.read_csv("../cenario1_engine.csv")

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3207 entries, 0 to 3206
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Brand_Mod         3207 non-null   object 
 1   model             3207 non-null   object 
 2   model_year        3207 non-null   int64  
 3   KM                3207 non-null   int64  
 4   fuel              3207 non-null   object 
 5   HP                2578 non-null   float64
 6   Litros            2891 non-null   float64
 7   Cilindros         2705 non-null   float64
 8   transmission_mod  3207 non-null   object 
 9   ext_col_mod       3207 non-null   object 
 10  int_col_mod       3207 non-null   object 
 11  accident_mod      3207 non-null   object 
 12  clean_title_mod   3207 non-null   object 
 13  price_eur         3207 non-null   int64  
dtypes: float64(3), int64(3), object(8)
memory usage: 350.9+ KB


In [17]:
df = df.dropna()

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2453 entries, 0 to 3206
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Brand_Mod         2453 non-null   object 
 1   model             2453 non-null   object 
 2   model_year        2453 non-null   int64  
 3   KM                2453 non-null   int64  
 4   fuel              2453 non-null   object 
 5   HP                2453 non-null   float64
 6   Litros            2453 non-null   float64
 7   Cilindros         2453 non-null   float64
 8   transmission_mod  2453 non-null   object 
 9   ext_col_mod       2453 non-null   object 
 10  int_col_mod       2453 non-null   object 
 11  accident_mod      2453 non-null   object 
 12  clean_title_mod   2453 non-null   object 
 13  price_eur         2453 non-null   int64  
dtypes: float64(3), int64(3), object(8)
memory usage: 287.5+ KB


In [19]:
categorical_columns = df.select_dtypes(include='object').columns

for column in categorical_columns:
    df[column] = df[column].astype('category').cat.codes

In [20]:
X, y = df.drop("price_eur", axis=1), df["price_eur"]

In [21]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=1234)

Random Forest

In [22]:
rf = RandomForestRegressor(random_state=1234)
scores_rf = cross_validate(rf, X, y, cv=splitter, scoring=METRICS_REGRESSION)
rf_scores = pd.DataFrame(scores_rf)
pd.DataFrame(rf_scores.mean()).T

,fit_time,score_time,test_MAE,test_MSE,test_RMSE,test_R2
0,2.185811,0.008949,10065.073371,4.088155e+09,42426.614174,0.473366


Redes Neuronais

In [23]:
nn = MLPRegressor(hidden_layer_sizes=(50,50), max_iter=200, random_state=1234)
scores_nn = cross_validate(nn, X, y, cv=splitter, scoring=METRICS_REGRESSION)
nn_scores = pd.DataFrame(scores_nn)
pd.DataFrame(nn_scores.mean()).T

,fit_time,score_time,test_MAE,test_MSE,test_RMSE,test_R2
0,1.686437,0.002402,17083.44181,4.263896e+09,47020.444869,0.284287


Support Vector Machine

In [24]:
svm = SVR()
scores_svm = cross_validate(svm, X, y, cv=splitter, scoring=METRICS_REGRESSION)
svm_scores = pd.DataFrame(scores_svm)
pd.DataFrame(svm_scores.mean()).T

,fit_time,score_time,test_MAE,test_MSE,test_RMSE,test_R2
0,0.16775,0.037004,21066.804947,4.912119e+09,54906.106366,-0.048246
